In [115]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore


In [ ]:
import os
api_key=os.getenv('Q_API_KEY')

In [117]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model_name="gpt-4o-mini", max_tokens=500)

In [118]:
# Carregar o PDF
pdf_link = "6x1.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)

pages = loader.load_and_split()

In [119]:
len(pages)

7

In [120]:
# Splitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size = 200)

parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000,
    chunk_overlap = 200,
    length_function = len,
    add_start_index = True
)

In [121]:
# Storages
store = InMemoryStore()
vectorstore = Chroma(embedding_function=embeddings, persist_directory='childVectorDB')

In [122]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

parent_document_retriever.add_documents(pages, ids=None)

In [123]:
# parent_document_retriever.vectorstore.get()

In [124]:
TEMPLATE = """
  Você é um especialista em legislação e em trabalho. Responda a pergunta abaixo utilizando o contexto informado.
  Query:
  {question}

  Context:
  {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [125]:
setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": parent_document_retriever})

output_parser = StrOutputParser()

In [126]:
parent_chain_retrieval = setup_retrieval | rag_prompt | llm | output_parser

In [127]:
parent_chain_retrieval.invoke("Com base no documento apresentado quais os principais riscos se for aprovado?")

'Com base no documento apresentado, os principais riscos associados à aprovação da emenda à constituição que propõe a redução da jornada de trabalho com a manutenção do salário incluem:\n\n1. **Ambiguidade na Implementação**: Apesar de a proposta buscar proteger os trabalhadores contra a redução indireta de remuneração, a falta de clareza na implementação pode levar a interpretações diversas, resultando em desigualdades e inseguranças trabalhistas.\n\n2. **Impacto nas Pequenas Empresas**: A obrigação de manter salários na redução da jornada pode representar um desafio financeiro para pequenas e médias empresas, que podem não ter a mesma flexibilidade financeira que grandes corporações, potencialmente levando a demissões ou fechamento de negócios.\n\n3. **Desigualdade na Aplicação**: A proposta pode beneficiar apenas setores que já possuem uma estrutura mais robusta para implementar tais mudanças, deixando outros setores vulneráveis a abusos e precarização do trabalho.\n\n4. **Possível 